In [ ]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.optimize import minimize
from pymoo.core.crossover import Crossover
from pymoo.operators.crossover.binx import BX
from pymoo.core.mutation import Mutation
from pymoo.core.sampling import Sampling
from pymoo.core.population import Population
from pymoo.core.evaluator import Evaluator
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter
import numpy as np 
import pandas as pd

class Portfolio_Problem(ElementwiseProblem):
    def __init__(self,N, profits, risk,esg, **kwargs):
        self.profits = profits
        self.risk = risk
        self.esg = esg
        super().__init__(n_var=N,
                         n_obj=2,
                         xl=0.0,
                         xu=1.0,
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        #NORMALIZAR X antes de empezar
        s = np.sum(x)
        x = x/s
        #Función de evaluacion
        exp_return  =   self.profits@x
        exp_risk    =   np.sqrt(x.T@self.risk@x)
        exp_esg     =   self.esg@x
        sharpe      =   exp_return / exp_risk
        out['F']    =   [exp_risk, -exp_return]
        out['ESG']  =   exp_esg
        out['Sharpe'] = sharpe
        
class Portfolio_Repair(Repair):
    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

In [ ]:
class MySampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        #100 individuos (a lo más 2000)
        #problem.n_var la cantidad de activos. 
        X = get_reference_directions("energy", problem.n_var, n_samples, seed=1)
        return X 
    
class MyCrossover(Crossover):
    def __init__(self):
        # define the crossover: number of parents and number of offsprings
        super().__init__(3, 1)

    def _do(self, problem, X, **kwargs):
        # The input of has the following shape (n_parents, n_matings, n_var)
        n_parents, n_matings, n_var = X.shape
        # The output owith the shape (n_offsprings, n_matings, n_var)
        # Because there the number of parents and offsprings are equal it keeps the shape of X
        _X = np.full(self.n_offsprings, n_matings, problem.n_var), 0)
        for k in range(n_matings): 
            x0, x1, x2 = X[0, k], X[1, k], X[2, k] 
            alpha = 0.8  #¿Mejor valor para alpha? 
            off_x0= x0 + alpha*(x2-x1)
            off_x0[off_x0<0]=0 #Reparar negativos
            off_x0= off_x0/np.sum(off_x0) #Reparar suma 
            _X[0, k] = off_x0
        
        #Falta un segundo crossover 
        return _X
    
class MyMutation(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):

        # for each individual
        for i in range(len(X)):

            r = np.random.random()

            # with a probabilty of 40% - change the order of characters
            if r < 0.4:
                perm = np.random.permutation(problem.n_characters)
                X[i, 0] = "".join(np.array([e for e in X[i, 0]])[perm])

            # also with a probabilty of 40% - change a character randomly
            elif r < 0.8:
                prob = 1 / problem.n_characters
                mut = [c if np.random.random() > prob
                       else np.random.choice(problem.ALPHABET) for c in X[i, 0]]
                X[i, 0] = "".join(mut)

        return X

In [3]:


ref_dirs = get_reference_directions("energy", 3, 400, seed=1)
ref_dirs

array([[0.        , 0.        , 1.        ],
       [0.        , 0.03916851, 0.96083149],
       [0.        , 0.07656481, 0.92343519],
       ...,
       [0.95966447, 0.        , 0.04033553],
       [0.96190939, 0.03809061, 0.        ],
       [1.        , 0.        , 0.        ]])

In [4]:
ref_dirs.shape

(400, 3)